# Dependency code

In [ ]:
import imp
import os
import sys
import numpy as np
import glob
import cmdbench

fp, pathname, description = imp.find_module('benchmark', ['../lib'])
benchmark = imp.load_module('benchmark', fp, pathname, description)

# Summarize numpy array if it has more than 10 elements
np.set_printoptions(threshold=10)

# Software versions

COBS will have to be built directly from source <https://github.com/bingmann/cobs>. Version used is commit `1915fc061bbe47946116b4a051ed7b4e3f3eca15`. I have included the binary in the directory `../lib/bin`.

In [ ]:
cobs_bin = '../lib/bin/cobs'
!ls ../lib/bin
!{cobs_bin}

# Benchmarking

## Input data and constants

In [ ]:
input_dir = '../data/input-files/reads'

input_files_1 = [os.path.basename(f) for f in glob.glob(f'{input_dir}/*_1.fastq.gz')]
input_samples = [f.replace('_1.fastq.gz','') for f in input_files_1]
input_samples.sort()
print(input_samples)

nproc=32
kmer=31
hashes=3

sample_sizes = [1,10,20,30,40,50,60,70,80]

benchmark.create_folder_if_doesnt_exist("cobs-data")
benchmark.create_folder_if_doesnt_exist("cobs-data/samples")

## Benchmark functions

In [ ]:
def sampling_func(sample_size):
    # Copy files we want to be part of the sample
    # from inputs to cobs-data/samples/ and return their path
    samples = input_samples[:sample_size]
    samples_out = []
    for sample in samples:
        infile1, infile2 = os.path.abspath(f"{input_dir}/{sample}_1.fastq.gz"), os.path.abspath(f"{input_dir}/{sample}_2.fastq.gz")
        outfile1, outfile2 = f"cobs-data/samples/{sample}_1.fastq.gz", f"cobs-data/samples/{sample}_2.fastq.gz"
        os.symlink(infile1, outfile1)
        os.symlink(infile2, outfile2)
    return ["cobs-data/samples/"]

def reset_func():
    benchmark.clean_if_exists("cobs-data/samples/")
    benchmark.clean_if_exists("cobs-data/example.cobs_compact")
    benchmark.clean_if_exists("cobs-data/example.cobs_compact.tmp")

build_command = {
    "command": f"{cobs_bin} compact-construct -k {kmer} --threads {nproc} --num-hashes {hashes} cobs-data/samples/ cobs-data/example.cobs_compact",
}

query_repeat_count = 2
query_sequence = "GAAGAAGATGGTGTACGCGGTGCGCGCCGCTATCTCGACCACCTTAAAATGGAATATGCCTTCTGGATGGACG"

query_command = {
    "command": f"{cobs_bin} query -i cobs-data/example.cobs_compact %",
    "parallel_argfiles": [query_sequence] * query_repeat_count,
    "parallel_args": f"-j {nproc} -I%"
}

## Benchmark

In [ ]:
multibench_results, debug_str = benchmark.multibench.multi_cmdbench({
        "index": [build_command],
        "query": [query_command]
    },
    reset_func = reset_func, iterations = 1, sampling_func = sampling_func, sample_sizes = sample_sizes, 
    benchmark_list_to_results=benchmark.benchmark_list_to_results, active_output_print = True
)

# Save and reload results

In [ ]:
save_path = "cobs-results.txt"

samples_per_sample_size = []
for sample_size in sample_sizes:
        samples_per_sample_size.append(input_samples[:sample_size])

benchmark.multibench.save_multibench_results(multibench_results, samples_per_sample_size, save_path)
multibench_results, samples_per_sample_size = benchmark.multibench.read_multibench_results(save_path)
print(samples_per_sample_size)

# Plot

In [ ]:
%matplotlib inline
import matplotlib.pyplot as plt
plt.style.use('seaborn-whitegrid')
import numpy as np
from pylab import rcParams
rcParams['figure.figsize'] = 15, 3

In [ ]:
# Indexing Plots
benchmark.multibench.plot_resources(multibench_results, sample_sizes, "index")

In [ ]:
# Querying Plots
benchmark.multibench.plot_resources(multibench_results, sample_sizes, "query")